In [2]:
import pandas as pd
import xarray as xr
import numpy as np

def get_variable_from_netcfd(df, lon, lat, var):
    """ Extract variable from netcdf file and interpolate to 30 min intervals """
    return (df[var].sel(longitude=lon, latitude = lat, method="nearest")
            .to_dataframe()
            .set_index("valid_time")[[var]]
            .resample("1800s")
            .interpolate())


nwp = xr.load_dataset("data/hres_1day_south_scotland_202101_202306.nc")

nwp["ws100"] = pow(pow(nwp.u100,2) + pow(nwp.v100,2), 0.5)
nwp["ws10"] = pow(pow(nwp.u10,2) + pow(nwp.v10,2), 0.5)


nwp['wa100'] = np.rad2deg(np.arctan2(nwp.v100, nwp.u100))
nwp['wa10'] = np.rad2deg(np.arctan2(nwp.v10, nwp.u10))

wf_capacity = pd.read_csv('data/Central Scotland Wind Farms.csv')
variables = ['ws100', 'ws10', 'wa100', 'wa10']

wind_data_features = pd.DataFrame()

for index, farm in wf_capacity.iterrows():
    lon, lat = farm[["Longitude", "Latitude"]]
    weather_list = []
    for var in variables:
        weather_list.append(get_variable_from_netcfd(nwp, lon, lat, var))
    weather = pd.concat(weather_list, axis=1)
    weather['turbine'] = farm['Name']
    wind_data_features = pd.concat([wind_data_features, weather])